In [32]:

import pandas as pd

In [ ]:

data_path = 'C:/LINCS_DATA/project/data/'
zscore_path = data_path + 'zscore_matrix_392.csv'
combo_path = data_path + 'LABELS/combo_design.csv'
drug_path = data_path + 'LABELS/drug.csv'
signature_path = data_path + 'best_signature_per_drug_final.csv'
label_path = data_path + 'LABELS/synergy_score.csv'


In [ ]:

zscore_df = pd.read_csv(zscore_path, index_col='sig_id')
combo_df = pd.read_csv(combo_path)
drug_df = pd.read_csv(drug_path)
sig_df = pd.read_csv(signature_path)
label_df = pd.read_csv(label_path)

In [ ]:


drug_map = dict(zip(drug_df['idDrug'], drug_df['name']))

In [ ]:

combo_df = combo_df.reset_index(drop=True)
combo_df['row_index'] = combo_df.index + 1  # لمطابقة idSynergy_Score

In [ ]:

combo_df['drug1_name'] = combo_df['idDrugA'].map(drug_map).str.lower().str.strip()
combo_df['drug2_name'] = combo_df['idDrugB'].map(drug_map).str.lower().str.strip()
sig_df['pert_iname'] = sig_df['pert_iname'].str.lower().str.strip()


In [ ]:

sig_map = dict(zip(sig_df['pert_iname'], sig_df['sig_id']))
combo_df['sig_id_1'] = combo_df['drug1_name'].map(sig_map)
combo_df['sig_id_2'] = combo_df['drug2_name'].map(sig_map)

In [ ]:


combo_df = combo_df.dropna(subset=['sig_id_1', 'sig_id_2'])

In [ ]:


label_map = dict(zip(label_df['idSynergy_Score'], label_df['zip']))
combo_df['zip'] = combo_df['row_index'].map(label_map)

In [ ]:


combo_df['label'] = combo_df['zip'].apply(lambda x: 1 if x > -0.5 else 0)

In [ ]:

combo_df = combo_df.dropna(subset=['label'])

In [ ]:

X = []
y = []

for _, row in combo_df.iterrows():
    sig1 = row['sig_id_1']
    sig2 = row['sig_id_2']
    if sig1 in zscore_df.index and sig2 in zscore_df.index:
        vec1 = zscore_df.loc[sig1].values
        vec2 = zscore_df.loc[sig2].values
        pair_vector = list(vec1) + list(vec2)
        X.append(pair_vector)
        y.append(int(row['label']))


In [ ]:

features_df = pd.DataFrame(X)
labels_df = pd.DataFrame(y, columns=['label'])

features_df.to_csv('C:/LINCS_DATA/project/features/pair_feature_matrix.csv', index=False)
labels_df.to_csv('C:/LINCS_DATA/project/features/pair_labels.csv', index=False)

print(f"✅ تم إنشاء {len(X)} زوج دواء وتخزينهم بنجاح.")


✅ تم إنشاء 490 زوج دواء وتخزينهم بنجاح.
